# Lesson 25: Integrating Multiple AI Agents with MCP

Throughout the course, we've built two agents: Nova for research and Brown for writing. Now it's time to integrate them into a unified system. In this lesson, we will explore how to use both the Nova research agent and the Brown writing workflow together by leveraging the Model Context Protocol (MCP).

We've already seen how each agent works as an MCP server in previous lessons. Nova exposes 11 tools for research tasks, while Brown provides 3 tools for article generation and editing. The beauty of MCP is that it makes integration straightforward. We'll explore two approaches:

1. **Multi-Server MCP Client**: A single MCP client that connects to multiple independent MCP servers.
2. **Composed MCP Server**: A single MCP client that connects to a single MCP server that composes multiple MCP servers together using FastMCP's composition features.

Both approaches have their use cases, and by the end of this lesson, you'll understand when to use each one.

Learning objectives:
- Learn how to connect an MCP client to multiple MCP servers simultaneously
- Understand how to use FastMCP's composition features to create a unified MCP server
- Compare multi-server client vs composed server approaches
- See the practical benefits of MCP for agent integration

## 1. Setup

First, we define some standard Magic Python commands to autoreload Python packages whenever they change:

In [1]:
%load_ext autoreload
%autoreload 2

### Import Key Packages

In [2]:
import nest_asyncio
nest_asyncio.apply() # Allow nested async usage in notebooks

## 2. Understanding Multi-Agent Orchestration: The MCP Approach

Before we dive into the technical implementation, let's understand the orchestration model we're using and why it matters.

### The Central LLM Orchestration Pattern

In this lesson, we're implementing what's known as a **Central LLM Orchestration** pattern. In this approach, a single, central LLM (e.g. the one powering your IDE assistant, like Claude in Cursor) has access to tools from multiple specialized agents. When you give it a task, the LLM dynamically decides which agent's tools to use based on the task requirements, maintaining a single conversation context and orchestrating the workflow by selecting the appropriate tools as needed.

This is fundamentally different from other orchestration patterns. A **Supervisor Agent** pattern would have one agent explicitly delegate entire sub-tasks to worker agents. A **Sequential Pipeline** would force agents to always execute in a fixed order. **Peer-to-Peer Communication** would allow agents to directly message each other. Our central LLM orchestration is simpler: the LLM acts as a intelligent tool selector rather than an explicit coordinator.

We'll learn more about these other patterns in another lesson.

### Why This Pattern Works Well with MCP

The Model Context Protocol makes this orchestration pattern particularly elegant. Both Nova and Brown expose their capabilities as MCP tools with clear descriptions, allowing the central LLM to discover all available tools through a single, standardized protocol. This unified tool interface eliminates the need for custom integration code: because both agents speak MCP, we don't need to write adapters or APIs. The client simply connects to both servers and aggregates their tools.

The LLM's natural reasoning ability handles the orchestration logic. For example, it intuitively understands that it should use Nova's research tools first, review the results, and then use Brown's writing tools with that research as input. There's no need to explicitly program this workflow or create complex state machines.

The flexibility is another key advantage. You can easily add or remove agents by simply updating the configuration file without touching any code. This pattern essentially treats specialized agents as "tool libraries" that a central reasoning engine can draw from as needed, making the system both modular and maintainable.

### The Rationale: Why Choose Central LLM Orchestration?

This orchestration pattern offers several compelling advantages that make it ideal for our use case of integrating Nova and Brown.

The most immediate benefit is **simplicity and maintainability**. All decision-making happens in one place—the central LLM, which means the workflow is transparent and easy to understand. You can see which tools the LLM chooses in real-time as it works through a task. There's no need for complex state management or inter-agent communication protocols, which reduces the cognitive overhead of understanding and debugging the system.

The pattern also enables **natural task decomposition**. The central LLM can break down complex requests on the fly without requiring predefined workflows. Even more importantly, it can adapt its strategy based on intermediate results. For example, if research reveals unexpected information, the LLM can adjust its writing approach accordingly. This adaptive behavior is especially valuable for human-in-the-loop workflows common in IDE environments. You can provide feedback at any point in the process, and the LLM incorporates it naturally without requiring explicit handoff protocols.

The central LLM maintains a **single, unified context window**, which means it can reference information from Nova's research when calling Brown's tools without requiring explicit data passing between agents. This avoids the notorious "siloed knowledge" problem, where critical information gets trapped in one agent's context and becomes unavailable to others who need it.

Perhaps most importantly for our specific use case, this pattern is **perfect for sequential, interdependent tasks**. Our workflow (research followed by writing) is inherently sequential, and the writing task depends heavily on research results. A central orchestrator can easily manage these dependencies because it sees the entire workflow and can make informed decisions about when to transition from research to writing.

### The Tradeoffs: Understanding the Limitations

While central LLM orchestration with MCP is powerful, it's important to understand its limitations and recognize when you might need a different approach.

The first limitation arises from **tool overload**. As the number of available tools grows beyond approximately 15-20, LLMs begin to struggle with reliable tool selection. They may choose suboptimal tools or miss relevant ones entirely. This degradation in performance is well-documented in the research on agent systems. Our system, with 14 tools total, is comfortably within the limit, but if you were to add many more specialized agents, you'd eventually hit this ceiling and need to consider a different pattern.

Another significant limitation is the pattern's inherently **sequential execution model**. If you need to research 50 companies simultaneously, a single LLM executing tools one at a time would be painfully inefficient. In such scenarios, a Supervisor-Worker pattern with parallel execution would be better.

The pattern also struggles with **complex inter-agent dependencies**. If agents need to negotiate with each other, engage in debate, or iteratively refine each other's work through back-and-forth exchanges, direct agent-to-agent communication would be more natural. Our pattern handles simple, linear dependencies well (Nova's output feeds into Brown) but complex multi-way interactions would become awkward and difficult to manage.

However, central LLM orchestration with MCP is the **right default choice** for most agent integration scenarios. It's simple, maintainable, and leverages the LLM's natural reasoning abilities without adding unnecessary complexity. You should only consider more elaborate patterns when you hit clear scaling limits (e.g. too many tools causing selection problems) or have fundamentally different requirements like massive parallelism or complex agent negotiations.

We'll learn more about these other patterns in another lesson.

### Integrating Nova and Brown

In the previous lessons, we built two specialized agents: Nova for research (which ingests article guidelines, performs web research, scrapes sources, and compiles comprehensive research files) and Brown for writing (which takes research and guidelines to generate, review, and edit articles with human-in-the-loop feedback). These agents were designed to work in sequence—Nova gathers the research, and Brown uses that research to write the article—but we've been running them separately. Now we'll orchestrate them together using the central LLM orchestration pattern. Because both agents are already exposed as MCP servers, integration is straightforward: we simply connect to both servers and let the central LLM decide which tools to use and when.

We'll explore two different approaches for achieving this integration.
1. The first is a **Multi-Server MCP Client**, where a single client connects directly to multiple independent MCP servers simultaneously—the client aggregates all tools from both servers and presents them to the LLM.
2. The second is a **Composed MCP Server**, where we create a new server that internally connects to both Nova and Brown, exposing their combined capabilities as a single unified endpoint.

Both approaches achieve the same goal but differ in their deployment and configuration patterns.

## 3. Approach 1: Multi-Server MCP Client

The first approach is to create an MCP client that connects to multiple MCP servers simultaneously. FastMCP's `Client` class supports this out of the box by accepting a configuration object that specifies multiple servers.

### 3.1 Multi-Server Configuration File

Let's look at the configuration file that defines both Nova and Brown servers.

Source: _lessons/agents_integration/mcp_client/mcp_servers_config.json_

```json
{
  "mcpServers": {
    "nova-research-agent": {
      "transport": "stdio",
      "command": "uv",
      "args": [
        "--directory",
        "/absolute/path/to/research_agent_part_2/mcp_server",
        "run",
        "-m",
        "src.server",
        "--transport",
        "stdio"
      ]
    },
    "brown-writing-workflow": {
      "transport": "stdio",
      "command": "uv",
      "args": [
        "--directory",
        "/absolute/path/to/writing_workflow",
        "run",
        "python",
        "-m",
        "brown.mcp.server"
      ]
    }
  }
}
```

This configuration tells the MCP client how to launch both servers. Each server:
- Has a unique name (`nova-research-agent`, `brown-writing-workflow`)
- Uses the `stdio` transport (communicates via stdin/stdout)
- Specifies the command and arguments to start the server

### 3.2 Creating the Multi-Server Client

Now let's see how the client code loads this configuration and connects to both servers.

Source: _lessons/agents_integration/mcp_client/src/client.py_

```python
import json
from pathlib import Path
from fastmcp import Client

# Load configuration from JSON file
config_path = Path("mcp_servers_config.json")
with open(config_path) as f:
    config = json.load(f)

server_names = list(config["mcpServers"].keys())
logging.info(f"Found {len(server_names)} MCP servers in configuration: {', '.join(server_names)}")

# Create a single client with multi-server configuration
client = Client(config)

# Connect and fetch capabilities from all servers
async with client:
    tools = await client.list_tools()
    resources = await client.list_resources()
    prompts = await client.list_prompts()
    
    logging.info(
        f"Total capabilities: {len(tools)} tools, {len(resources)} resources, {len(prompts)} prompts"
    )
```

The key insight here is that `Client(config)` accepts a multi-server configuration. When you call `list_tools()`, `list_resources()`, or `list_prompts()`, the client automatically aggregates capabilities from all connected servers.


### 3.3 How Capabilities Are Named

When you have multiple servers, how do you distinguish which tool belongs to which server? FastMCP handles this by prefixing the tool names with the server name.

For example:
- Nova's `extract_guidelines_urls` tool becomes `nova-research-agent_extract_guidelines_urls`
- Brown's `generate_article` tool becomes `brown-writing-workflow_generate_article`

The client code groups capabilities by extracting these prefixes:

Source: _lessons/agents_integration/mcp_client/src/utils/command_utils.py_

```python
def handle_command(processed_input, tools, resources, prompts, server_names):
    # Extract server prefixes from tool names
    server_prefixes = set()
    for tool in tools:
        if "_" in tool.name:
            prefix = tool.name.split("_")[0]
            server_prefixes.add(prefix)
    
    # Group tools by prefix
    if processed_input.input_type == InputType.COMMAND_INFO_TOOLS:
        for prefix in sorted(server_prefixes):
            prefix_tools = [t for t in tools if t.name.startswith(f"{prefix}_")]
            if prefix_tools:
                print_header(f"{prefix} - Tools ({len(prefix_tools)})")
                for i, tool in enumerate(prefix_tools, 1):
                    print_item(tool.name, tool.description, i)
```

This approach makes it easy to see which capabilities come from which server.


### 3.4 Running the Multi-Server Client

When you run the multi-server client, you'll see output like this:

```bash
$ cd agents_integration/mcp_client
$ uv run -m src.client
```

```terminal
INFO:root:Loading MCP server configuration from: mcp_servers_config.json
INFO:root:Found 2 MCP servers in configuration: nova-research-agent, brown-writing-workflow
INFO:root:Connecting to MCP servers...
INFO:root:Fetching capabilities from all servers...
INFO:root:Total capabilities: 14 tools, 4 resources, 4 prompts

============================================================
Brown Writing Workflow
============================================================

  - 3 tools available
  - 2 resources available
  - 3 prompts available

============================================================
Nova Research Agent
============================================================

  - 11 tools available
  - 2 resources available
  - 1 prompts available

Available Commands: /tools, /resources, /prompts, /quit

>
```

When you type `/tools`, you'll see all tools from both servers:

```terminal
============================================================
brown-writing-workflow - Tools (3)
============================================================

1. brown-writing-workflow_generate_article
   Generate an article from scratch using Brown's article generation workflow.

2. brown-writing-workflow_edit_article
   Edit an entire article based on human feedback and expected requirements.

3. brown-writing-workflow_edit_selected_text
   Edit a selected section of an article based on human feedback.

============================================================
nova-research-agent - Tools (11)
============================================================

1. nova-research-agent_extract_guidelines_urls
   Extract URLs and local file references from article guidelines.

2. nova-research-agent_process_local_files
   Process local files referenced in the article guidelines.

3. nova-research-agent_scrape_and_clean_other_urls
   Scrape and clean other URLs from GUIDELINES_FILENAMES_FILE.

... (and 8 more tools)
```

This demonstrates that the client successfully connected to both servers and aggregated their capabilities.


### 3.5 Running the Multi-Server Client in the Notebook

Now let's run the multi-server client directly from this notebook. The client will connect to both Nova and Brown servers and display their capabilities.

*Note*: The client is interactive, so you can type commands like `/tools`, `/resources`, `/prompts`, or `/quit` when prompted. Type `/quit` to exit the client.

In [ ]:
import sys

from agents_integration.mcp_client.src.client import main as client_main


async def run_client():
    _argv_backup = sys.argv[:]
    sys.argv = ["client", "--config", "mcp_servers_config.json"]
    try:
        await client_main()
    finally:
        sys.argv = _argv_backup


# Start client with in-memory server
await run_client()

After running this cell, you should see:
1. Both servers starting up (Nova Research MCP Server and Brown MCP Server)
2. The total capabilities summary (14 tools, 4 resources, 4 prompts)
3. A welcome message for each server showing their individual capabilities
4. An interactive prompt where you can type commands

Try typing:
- `/tools` to see all tools from both servers
- `/resources` to see all resources
- `/prompts` to see all prompts
- `/quit` to exit

## 4. Approach 2: Composed MCP Server

The second approach is to create a new MCP server that composes the Nova and Brown servers together. Instead of the client connecting to multiple servers, you create a single composed server that internally proxies requests to the underlying servers.

This approach is useful when you want to:
- Package multiple agents as a single deployable unit
- Simplify the client-side configuration (client only needs to know about one server)
- Add a layer of coordination or orchestration between agents


### 4.1 Server Composition Configuration

First, we define which servers to compose:

Source: _lessons/agents_integration/mcp_server/mcp_servers_to_compose.json_

```json
{
  "mcpServers": {
    "nova-research-agent": {
      "transport": "stdio",
      "command": "uv",
      "args": [
        "--directory",
        "/absolute/path/to/research_agent_part_2/mcp_server",
        "run",
        "-m",
        "src.server",
        "--transport",
        "stdio"
      ]
    },
    "brown-writing-workflow": {
      "transport": "stdio",
      "command": "uv",
      "args": [
        "--directory",
        "/absolute/path/to/writing_workflow",
        "run",
        "python",
        "-m",
        "brown.mcp.server"
      ]
    }
  }
}
```

This looks identical to the multi-server client config, but it's used differently.


### 4.2 Creating the Composed Server

Now let's see how to create a composed server using FastMCP's composition features.

Source: _lessons/agents_integration/mcp_server/src/main.py_

```python
import json
import logging
from pathlib import Path
from fastmcp import Client, FastMCP

def load_server_config() -> dict:
    """Load the MCP servers configuration from JSON file."""
    config_path = Path(__file__).parent.parent / "mcp_servers_to_compose.json"
    with open(config_path) as f:
        return json.load(f)

def create_composed_server() -> FastMCP:
    """Create a composed MCP server by mounting Nova and Brown servers."""
    # Create the main composed server
    mcp = FastMCP(
        name="Nova+Brown Composed Server",
        version="0.1.0",
    )
    
    # Load configuration
    config = load_server_config()
    servers_config = config.get("mcpServers", {})
    
    # Create proxies and mount each server
    for server_name, server_config in servers_config.items():
        # Wrap the server config in the structure expected by Client
        client_config = {"mcpServers": {server_name: server_config}}
        
        # Create a client for this server
        client = Client(client_config)
        
        # Create a proxy from the client
        proxy = FastMCP.as_proxy(client)
        
        # Extract prefix: nova-research-agent -> nova
        prefix = server_name.split("-")[0]
        
        # Mount the proxy with the prefix
        mcp.mount(proxy, prefix=prefix)
    
    return mcp

if __name__ == "__main__":
    composed_server = create_composed_server()
    composed_server.run()
```

Let's break down the key steps:

1. First we create a FastMCP instance. This is our composed server.
2. For each server to compose:
   - Create a `Client` that connects to that server
   - Use `FastMCP.as_proxy(client)` to create a proxy object
   - Use `mcp.mount(proxy, prefix=prefix)` to mount it with a prefix

The `mount()` method is the magic here. It takes all capabilities from the proxy and adds them to the composed server with the specified prefix. This is how `extract_guidelines_urls` becomes `nova_extract_guidelines_urls`.


### 4.3 Running the Composed Server

To use the composed server, you need a client config that points to it:

Source: _lessons/agents_integration/mcp_client/mcp_composed_server_config.json_

```json
{
  "mcpServers": {
    "nova-brown-composed": {
      "transport": "stdio",
      "command": "uv",
      "args": [
        "--directory",
        "/absolute/path/to/agents_integration/mcp_server",
        "run",
        "python",
        "-m",
        "src.main"
      ]
    }
  }
}
```

Now when you run the client with this config:

```bash
$ cd agents_integration/mcp_client
$ uv run -m src.client --config mcp_composed_server_config.json
```

You'll see:

```terminal
INFO:root:Loading MCP server configuration from: mcp_composed_server_config.json
INFO:root:Found 1 MCP servers in configuration: nova-brown-composed
INFO:root:Connecting to MCP servers...
INFO:__main__:Starting composed MCP server...
INFO:__main__:Loading server configuration...
INFO:__main__:Found 2 servers to compose: ['nova-research-agent', 'brown-writing-workflow']
INFO:__main__:Creating proxy for nova-research-agent...
INFO:__main__:Mounting nova-research-agent with prefix 'nova'...
INFO:__main__:Creating proxy for brown-writing-workflow...
INFO:__main__:Mounting brown-writing-workflow with prefix 'brown'...
INFO:__main__:Composed server created successfully!
INFO:__main__:Running composed server...

INFO:root:Fetching capabilities from all servers...
INFO:root:Total capabilities: 14 tools, 4 resources, 4 prompts

============================================================
Brown
============================================================

  - 3 tools available
  - 2 resources available
  - 3 prompts available

============================================================
Nova
============================================================

  - 11 tools available
  - 2 resources available
  - 1 prompts available

Available Commands: /tools, /resources, /prompts, /quit
```

Notice the difference in prefixes:
- Multi-server client: `nova-research-agent_`, `brown-writing-workflow_`
- Composed server: `nova_`, `brown_`

This is because the composed server uses cleaner prefixes specified in the `mount()` call.

### 4.4 Listing Tools from the Composed Server

When you type `/tools`, you'll see:

```terminal
============================================================
brown - Tools (3)
============================================================

1. brown_generate_article
   Generate an article from scratch using Brown's article generation workflow.

2. brown_edit_article
   Edit an entire article based on human feedback.

3. brown_edit_selected_text
   Edit a selected section of an article based on human feedback.

============================================================
nova - Tools (11)
============================================================

1. nova_extract_guidelines_urls
   Extract URLs and local file references from article guidelines.

2. nova_process_local_files
   Process local files referenced in the article guidelines.

3. nova_scrape_and_clean_other_urls
   Scrape and clean other URLs from GUIDELINES_FILENAMES_FILE.

... (and 8 more tools)
```

From the client's perspective, it's connecting to a single server (`nova-brown-composed`), but that server internally proxies to both Nova and Brown.


### 4.5 Running the Composed Server Client in the Notebook

Now let's run the client with the composed server configuration. This time, the client connects to a single composed server that internally proxies to both Nova and Brown.

*Note*: This is also interactive. Type `/quit` to exit when you're done exploring.

In [ ]:
import sys

from agents_integration.mcp_client.src.client import main as client_main


async def run_client():
    _argv_backup = sys.argv[:]
    sys.argv = ["client", "--config", "mcp_composed_server_config.json"]
    try:
        await client_main()
    finally:
        sys.argv = _argv_backup


# Start client with in-memory server
await run_client()

After running this cell, you should see:
1. The composed server starting (Nova+Brown Composed Server)
2. Log messages showing the composition process (creating proxies, mounting servers)
3. Both Nova and Brown servers starting up in the background
4. The total capabilities summary (same 14 tools, 4 resources, 4 prompts)
5. Welcome messages with cleaner prefixes ("Brown" and "Nova" instead of full server names)
6. An interactive prompt for commands

Notice the differences compared to the multi-server client:
- Only one server name in the config (`nova-brown-composed`)
- Cleaner tool prefixes (`nova_` and `brown_` instead of `nova-research-agent_` and `brown-writing-workflow_`)
- Extra log messages showing the composition process

Try the same commands:
- `/tools` to see tools with cleaner prefixes
- `/resources` to see resources
- `/prompts` to see prompts
- `/quit` to exit

## 5. Multi-Server Client vs Composed Server: When to Use Each

Both approaches achieve the same goal: using Nova and Brown together. But they have different use cases and trade-offs.

| Aspect | Multi-Server Client | Composed Server |
|--------|---------------------|-----------------|
| **How it works** | The client connects to multiple independent servers simultaneously | A single server internally proxies to multiple underlying servers |
| **Process management** | Each server runs independently with its own process | Single server process |
| **Client configuration** | Client needs to know about all servers | Single endpoint (simpler client config) |
| **Flexibility** | Easy to add/remove servers without changing code | Requires code changes to modify composition |
| **Coordination** | No opportunity for server-side coordination | Opportunity to add coordination logic between agents |
| **Deployment** | Good for development and testing | Better for production deployments |
| **Fault tolerance** | One server can fail without affecting others | If composed server fails, all agents become unavailable |
| **Use when** | • Developing or testing agents<br>• Quickly combining existing agents<br>• Need flexibility to add/remove agents dynamically | • Deploying agents as a unified system<br>• Need coordination logic between agents<br>• Want simpler client experience<br>• Building a product that packages multiple agents |
| **Practical example** | **Development**: Use while building and testing Nova and Brown separately. Easy to restart one server without affecting the other | **Production**: Deploy and use from Cursor or Claude Desktop. Users configure one MCP server and get access to both agents |

## 6. Conclusion

In this lesson, we explored how to integrate the Nova research agent and Brown writing workflow using MCP. We learned:

1. **Multi-Server MCP Client**: How to connect a single client to multiple MCP servers using FastMCP's multi-server configuration
2. **Composed MCP Server**: How to use FastMCP's composition features (`as_proxy()` and `mount()`) to create a unified server
4. **Use Cases**: When to use multi-server client vs composed server

The key insight is that MCP makes agent integration straightforward. Because both Nova and Brown are already MCP servers, we don't need to write custom integration code. We simply leverage MCP's standardized protocol and FastMCP's composition features.

This lesson focused on the technical mechanics of integration. In a real-world scenario, you would use these integrated agents within an IDE like Cursor. You could:
- Use Nova to research a topic
- Review the research file
- Use Brown to generate an article from that research
- Iterate on the article with Brown's editing tools
- All from within your editor, with beautiful diff views and human-in-the-loop feedback

In the next lesson, you'll see how to use both agents from Cursor to work on an article end-to-end. In Part 3 of this course, we'll explore production deployment, including how to deploy these composed servers remotely, add monitoring with Opik, and implement security measures.